In [1]:
from src.constants import *
from src.model.my_model import MyModel
from tqdm import  tqdm
from monai.networks import one_hot
from src.data.data_loaders import get_loaders
from src.utils.metrics import dice_scores
import numpy as np


<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
<class 'monai.transforms.utility.array.AsChannelFirst'>: Class `AsChannelFirst` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.


In [2]:
train_loader, test_loader = get_loaders()


Loading dataset: 100%|██████████| 70/70 [07:25<00:00,  6.37s/it]
IOStream.flush timed out
Loading dataset: 100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


In [3]:

def train_single_epoch(model, optimizer, train_loader, loss_fn):
    losses = []

    for d in train_loader:

        img = d['img'].to(DEVICE)
        mask = d['mask'].to(DEVICE)

        optimizer.zero_grad()

        outputs = model(img)

        outputs = torch.softmax(outputs, dim=1)
        mask = one_hot(mask, num_classes=3)

        loss = loss_fn(outputs, mask)
        loss.backward()
        optimizer.step()

        losses.append(loss.detach().cpu().item())
    print((sum(losses) / len(losses)))
    return model

In [4]:
def test_single_epoch(model, test_loader):
    model.eval()
    scores = []

    largest_component = monai.transforms.KeepLargestConnectedComponent()

    visualised = False

    with torch.no_grad():
        for d in test_loader:
            img = d['img'].to(DEVICE)
            mask = d['mask'].to(DEVICE)

            out = monai.inferers.sliding_window_inference(img,
                                                          roi_size=CROP_SIZE,
                                                          sw_batch_size=BATCH_SIZE,
                                                          predictor=model,
                                                          overlap=0.5,
                                                          sw_device=DEVICE,
                                                          device="cpu",
                                                          progress=False,
                                                          )
            out = torch.argmax(out, 1, keepdim=True)
            out = largest_component(out).to(DEVICE)
            s = dice_scores(out, mask)
            print(s)
            scores.append(s)

        scores = np.array(scores)
        scores = np.nan_to_num(scores, copy=True, nan=1.0)
        scores = np.sum(scores, axis=0) / scores.shape[0]

    test_score = np.sum(scores) / (len(scores) * 1.0)

In [7]:
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=16,
                 big_channel=3,
                 patch_size=8,
                 embed_dim=128,
                 skip_transformer=True).cuda()

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fn = monai.losses.DiceCELoss()

In [8]:
for e in tqdm(range(50)):
    train_single_epoch(model, optimizer, train_loader, loss_fn)

  2%|▏         | 1/50 [00:20<16:51, 20.64s/it]

1.7891828298568726


  4%|▍         | 2/50 [00:40<16:03, 20.07s/it]

1.7645972899028233


  6%|▌         | 3/50 [00:59<15:33, 19.85s/it]

...

1.736438420840672


 20%|██        | 10/50 [03:09<12:02, 18.06s/it]

1.7355221441813877


 20%|██        | 10/50 [03:19<13:19, 19.98s/it]


KeyboardInterrupt: 

In [ ]:
%%time
test_single_epoch(model=model, test_loader=test_loader)

In [2]:
d =  next(iter(train_loader))

NameError: name 'train_loader' is not defined

In [1]:
d['img'].shape

NameError: name 'd' is not defined